In [17]:
import json
# a simple filter that prepends line numbers
# import sys EDIT: unused, pointed out in comments here and on HN
for fname in ( '/Users/chathura/Documents/Git/yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_review.json',):
    with open(fname, 'r+') as f:
        lineno = 0
        # this reads in one line at a time from stdin
        for line in f:
            lineno += 1
            #print '{:>6} {}'.format(lineno, line)
            j = json.loads(line)
            print j['user_id']
            print j['review_id']
            print j['business_id']
            print j['stars']
            print j['text']
            print j['votes']
            print "---"
            
            if lineno == 100 :
                break

Xqd0DzHaiyRqVH3WRG7hzg
15SdjuK7DmYqUAj6rjGowg
vcNAWiLM4dR7D2nwwJ7nCA
5
dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank.
{u'funny': 0, u'useful': 2, u'cool': 1}
---
H1kH6QZV7Le4zqTRNxoZow
RF6UnRTtG7tWMcrO2GEoAg
vcNAWiLM4dR7D2nwwJ7nCA
2
Unfortunately, the frustration of being Dr. Goldberg's patient is a repeat of the experience I've had with so many other doctors in NYC -- good doctor, terrible staff.  It seems that his staff simply never answers the phone.  It usually takes 2 hours of repeated calling to get a

In [3]:
import json
# a simple filter that prepends line numbers
# import sys EDIT: unused, pointed out in comments here and on HN
for fname in ( '/Users/chathura/Documents/Git/yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_tip.json',):
    with open(fname, 'r+') as f:
        lineno = 0
        # this reads in one line at a time from stdin
        for line in f:
            lineno += 1
            print '{:>6} {}'.format(lineno, line)
            #j = json.loads(line)
            ##print j['user_id']
            #print j['review_id']
            #print j['business_id']
            #print j['stars']
            #print j['text']
            #print j['votes']
            #print "---"
            
            if lineno == 100 :
                break

     1 {"user_id": "-6rEfobYjMxpUWLNxszaxQ", "text": "Don't waste your time.", "business_id": "cE27W9VPgO88Qxe4ol6y_g", "likes": 0, "date": "2013-04-18", "type": "tip"}

     2 {"user_id": "EZ0r9dKKtEGVx2CdnowPCw", "text": "Your GPS will not allow you to find this place. Put Rankin police department in instead. They are directly across the street.", "business_id": "mVHrayjG3uZ_RLHkLj-AMg", "likes": 0, "date": "2013-01-06", "type": "tip"}

     3 {"user_id": "xb6zEQCw9I-Gl0g06e1KsQ", "text": "Great drink specials!", "business_id": "KayYbHCt-RkbGcPdGOThNg", "likes": 0, "date": "2013-12-03", "type": "tip"}

     4 {"user_id": "fvTivrsJoUMYXnOJw9wZfw", "text": "Sarah rocks! Best waitress here! Be sure to get her and compliment her glasses.", "business_id": "wJr6kSA5dchdgOdwH6dZ2w", "likes": 0, "date": "2013-07-22", "type": "tip"}

     5 {"user_id": "6GrH6gp09pqYykGv86D6Dg", "text": "Decent selection of more than just bar food. \n\nJumbo fish sandwich is aptly named. \n\nSeem to be generou

Tips mining for businesses: location related tips for rastarutens in busy suberbs

Tip :
{"user_id": "KoplwVOqt46ON4MWnTb_4g", "text": "There is a lot of parking in the back of you can't find a space in the front.", "business_id": "IqbY-ztC3GC_A_KoKhYKuA", "likes": 0, "date": "2014-12-02", "type": "tip"}


Review :
{"votes": {"funny": 5, "useful": 7, "cool": 3}, "user_id": "m7vtKWpZ9wdEQ95wJxrMrg", "review_id": "czWOtAkRhO4P90rOKAvRbA", "stars": 5, "date": "2014-11-24", "text": "Good, inexpensive Thai cuisine, well presented and no corners cut even as they keep their prices down. The service is friendly and the place is cozy and conveniently located. While parking in front of this commercial strip facing St. Joe's hospital is very limited, there is ample parking behind - as is known by those who frequent the popular 5th Avenue Cafe which happens to be their neighbor.", "type": "review", "business_id": "IqbY-ztC3GC_A_KoKhYKuA"}

Issue: need to identify a busy street based on businesses around it. may be to find the busiest time of the day and find a tip according to the situation.